## EDA

In [1]:
import pandas as pd

df = pd.read_csv('Dataset.csv')
# Concat Subject, Age, Handedness and Sex as strings and store in uuid column, all separated by hyphen
df['uuid'] = df['Subject'].astype(str) + '-' + df['Age'].astype(str) + '-' + df['Handedness'].astype(str) + '-' + df['Sex'].astype(str)
df.head()

,Subject,Age,Handedness,Sex,c1,c2,c3,c4,c5,l1,...,l3,l4,l5,TrialType,Trial_Num,Trial,TragetSlide.RT,GROUP,TargetSlide.ACC,uuid
0,11.0,26.0,right,female,lime,red,red,lime,lime,x,...,o,o,x,1.0,NaN,Prac,5045.0,1.0,0.0,11.0-26.0-right-female
1,11.0,26.0,right,female,red,lime,lime,lime,red,o,...,o,x,o,1.0,NaN,Prac,971.0,1.0,0.0,11.0-26.0-right-female
2,11.0,26.0,right,female,lime,red,red,red,lime,x,...,o,x,o,2.0,NaN,Prac,2411.0,1.0,0.0,11.0-26.0-right-female
3,11.0,26.0,right,female,red,red,lime,red,lime,o,...,x,o,x,1.0,NaN,Prac,1242.0,1.0,1.0,11.0-26.0-right-female
4,11.0,26.0,right,female,lime,lime,red,red,red,x,...,x,x,o,2.0,1.0,Exp,1654.0,1.0,0.0,11.0-26.0-right-female


### RT vs ADHD

In [62]:
adhd_rt = df[df['GROUP']==1]['TragetSlide.RT'].tolist()
non_adhd_rt = df[df['GROUP']==0]['TragetSlide.RT'].tolist()

# Perform statistical test to determine if their means are significantly different, at 95% confidence level. Print the result
import scipy.stats as stats
statistic, p_val = stats.ttest_ind(adhd_rt, non_adhd_rt)
print(f'Statistic: {statistic}, p-value: {p_val}')

if p_val < 0.05:
    print('Means are significantly different')
else:
    print('Means are not significantly different')

# Print the mean of the two groups
print(f'Mean of ADHD_RT: {sum(adhd_rt)/len(adhd_rt)}')
print(f'Mean of Non_ADHD_RT: {sum(non_adhd_rt)/len(non_adhd_rt)}')

Statistic: 15.111762245476397, p-value: 2.8660739006357743e-50
Means are significantly different
Mean of ADHD_RT: 3353.567
Mean of Non_ADHD_RT: 2465.4412037037036


### Practices vs RT

In [44]:
res = df.groupby(['uuid', 'GROUP', 'Trial']).agg({'Trial':'count'}).rename(columns={'Trial':'Count'}).reset_index()
res

,uuid,GROUP,Trial,Count
0,10.0-27.0-right-male,0.0,Exp,64
1,10.0-27.0-right-male,0.0,Prac,16
2,11.0-26.0-right-female,1.0,Exp,76
3,11.0-26.0-right-female,1.0,Prac,4
4,11.0-30.0-left-male,0.0,Exp,64
...,...,...,...,...
99,8.0-24.0-right-female,0.0,Prac,16
100,9.0-26.0-left-female,0.0,Exp,64
101,9.0-26.0-left-female,0.0,Prac,16
102,93.0-24.0-right-female,0.0,Exp,76


In [45]:
uuid_76_exp = list(res[(res['Trial']=='Prac') & (res['Count']==4)]['uuid'])
len(uuid_76_exp)

26

In [46]:
df['76exps'] = [1 if uuid in uuid_76_exp else 0 for uuid in df['uuid']]
df.head()

,Subject,Age,Handedness,Sex,c1,c2,c3,c4,c5,l1,...,l4,l5,TrialType,Trial_Num,Trial,TragetSlide.RT,GROUP,TargetSlide.ACC,uuid,76exps
0,11.0,26.0,right,female,lime,red,red,lime,lime,x,...,o,x,1.0,NaN,Prac,5045.0,1.0,0.0,11.0-26.0-right-female,1
1,11.0,26.0,right,female,red,lime,lime,lime,red,o,...,x,o,1.0,NaN,Prac,971.0,1.0,0.0,11.0-26.0-right-female,1
2,11.0,26.0,right,female,lime,red,red,red,lime,x,...,x,o,2.0,NaN,Prac,2411.0,1.0,0.0,11.0-26.0-right-female,1
3,11.0,26.0,right,female,red,red,lime,red,lime,o,...,o,x,1.0,NaN,Prac,1242.0,1.0,1.0,11.0-26.0-right-female,1
4,11.0,26.0,right,female,lime,lime,red,red,red,x,...,x,o,2.0,1.0,Exp,1654.0,1.0,0.0,11.0-26.0-right-female,1


In [61]:
df.groupby(['uuid', '76exps']).agg({'TragetSlide.RT':'mean'}).reset_index()
rt_76exp = df[df['76exps']==1]['TragetSlide.RT'].tolist()
rt_64exp = df[df['76exps']==0]['TragetSlide.RT'].tolist()

# Perform statistical test to determine if their means are significantly different, at 95% confidence level. Print the result
import scipy.stats as stats
statistic, p_val = stats.ttest_ind(rt_76exp, rt_64exp)
print(f'Statistic: {statistic}, p-value: {p_val}')

if p_val < 0.05:
    print('Means are significantly different')
else:
    print('Means are not significantly different')

# Print the mean of the two groups
print(f'Mean of 76exp: {sum(rt_76exp)/len(rt_76exp)}')
print(f'Mean of 64exp: {sum(rt_64exp)/len(rt_64exp)}')

Statistic: 15.49462299846886, p-value: 1.0920802645500971e-52
Means are significantly different
Mean of 76exp: 3346.7947115384613
Mean of 64exp: 2438.054807692308


### ADHD vs Age

In [67]:
agg_df = df.groupby(['uuid', 'Age', 'GROUP']).agg({'TragetSlide.RT':'mean'}).reset_index()

adhd_age = agg_df[agg_df['GROUP']==1]['Age'].tolist()
non_adhd_age = agg_df[agg_df['GROUP']==0]['Age'].tolist()

# Perform statistical test to determine if their means are significantly different, at 95% confidence level. Print the result
import scipy.stats as stats
statistic, p_val = stats.ttest_ind(adhd_age, non_adhd_age)
print(f'Statistic: {statistic}, p-value: {p_val}')

if p_val < 0.05:
    print('Means are significantly different')
else:
    print('Means are not significantly different')

# Print the mean of the two groups
print(f'Mean of ADHD_Age: {sum(adhd_age)/len(adhd_age)}')
print(f'Mean of Non_ADHD_Age: {sum(non_adhd_age)/len(non_adhd_age)}')

Statistic: 1.688556831493161, p-value: 0.09753267292792009
Means are not significantly different
Mean of ADHD_Age: 26.88
Mean of Non_ADHD_Age: 25.555555555555557


## ML

In [2]:
import pandas as pd

original_data = pd.read_csv('Dataset.csv')
original_data.head()

,Subject,Age,Handedness,Sex,c1,c2,c3,c4,c5,l1,l2,l3,l4,l5,TrialType,Trial_Num,Trial,TragetSlide.RT,GROUP,TargetSlide.ACC
0,11.0,26.0,right,female,lime,red,red,lime,lime,x,o,o,o,x,1.0,NaN,Prac,5045.0,1.0,0.0
1,11.0,26.0,right,female,red,lime,lime,lime,red,o,x,o,x,o,1.0,NaN,Prac,971.0,1.0,0.0
2,11.0,26.0,right,female,lime,red,red,red,lime,x,x,o,x,o,2.0,NaN,Prac,2411.0,1.0,0.0
3,11.0,26.0,right,female,red,red,lime,red,lime,o,o,x,o,x,1.0,NaN,Prac,1242.0,1.0,1.0
4,11.0,26.0,right,female,lime,lime,red,red,red,x,o,x,x,o,2.0,1.0,Exp,1654.0,1.0,0.0


In [4]:
# Rename columns and drop unnecessary columns
df = original_data.rename(columns={'TargetSlide.ACC ': 'TargetSlide.ACC'})
df = df.drop(columns=['c1', 'c2', 'c3', 'c4', 'c5', 'l1', 'l2', 'l3', 'l4', 'l5'])

# Convert columns to integers
for column in ['Subject', 'Age', 'TrialType', 'TragetSlide.RT', 'GROUP', 'TargetSlide.ACC']:
    df[column] = df[column].astype(int)
df['Trial_Num'] = df['Trial_Num'].fillna(0).astype(int)

# Convert columns to categorical
df['Handedness'] = df['Handedness'].replace({'right':0, 'left':1})
df['Sex'] = df['Sex'].replace({'male':0, 'female':1})
df['TrialType'] = df['TrialType'].replace({1:0, 2:1})
df['Trial'] = df['Trial'].replace({'Prac':0, 'Exp':1})

# print the mean and std
print(f'TargetSlide.RT mean: {df["TragetSlide.RT"].mean()}')
print(f'TargetSlide.RT std: {df["TragetSlide.RT"].std()}')
print(f'Age mean: {df["Age"].mean()}')
print(f'Age std: {df["Age"].std()}')

# Normalize continuous features if needed
df['TragetSlide.RT'] = (df['TragetSlide.RT'] - df['TragetSlide.RT'].mean()) / df['TragetSlide.RT'].std()
df['Age'] = (df['Age'] - df['Age'].mean()) / df['Age'].std()


# Concat Subject, Age, Handedness and Sex as strings and store in uuid column
df['patient_id'] = df['Subject'].astype(str) + df['Age'].astype(str) + df['Handedness'].astype(str) + df['Sex'].astype(str)

df.head()

TargetSlide.RT mean: 2892.4247596153846
TargetSlide.RT std: 1944.9682754013772
Age mean: 26.192307692307693
Age std: 2.849353530834194


,Subject,Age,Handedness,Sex,TrialType,Trial_Num,Trial,TragetSlide.RT,GROUP,TargetSlide.ACC,patient_id
0,11,-0.067492,0,1,0,0,0,1.106741,1,0,11-0.0674916924932766201
1,11,-0.067492,0,1,0,0,0,-0.987895,1,0,11-0.0674916924932766201
2,11,-0.067492,0,1,1,0,0,-0.247523,1,0,11-0.0674916924932766201
3,11,-0.067492,0,1,0,0,0,-0.848561,1,1,11-0.0674916924932766201
4,11,-0.067492,0,1,1,1,1,-0.636733,1,0,11-0.0674916924932766201


### LSTM

In [3]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

feature_columns = ['Age', 'Sex', 'Handedness', 'TrialType', 'TragetSlide.RT', 'TargetSlide.ACC']
features = df[feature_columns]
labels = df['GROUP']

def get_split_df(df, test_size=0.25, random_state=0):
    # Get unique patient IDs from df, assign random patient IDs to train and test sets
    patient_ids = df['patient_id'].unique()
    train_ids, test_ids = train_test_split(patient_ids, test_size=test_size, random_state=random_state)
    train_df = df[df['patient_id'].isin(train_ids)]
    test_df = df[df['patient_id'].isin(test_ids)]
    return train_df, test_df

class ADHDataset(Dataset):
    def __init__(self, data, feature_columns):
        self.data = data
        self.feature_columns = feature_columns
        self.patients = data['patient_id'].unique()
        
    def __len__(self):
        return len(self.patients)
    
    def __getitem__(self, idx):
        patient_id = self.patients[idx]
        patient_data = self.data[self.data['patient_id'] == patient_id]
        features = patient_data[self.feature_columns].values
        label = patient_data['GROUP'].values[0]  # Same for all rows of a patient
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

def get_dataloaders(train_df, test_df, feature_columns, batch_size=32):
    train_dataset = ADHDataset(train_df, feature_columns)
    test_dataset = ADHDataset(test_df, feature_columns)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class ADHDModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(ADHDModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 2)  # Binary classification
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)
        return out

In [5]:
def get_metrics(model, loader, debug=False):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for features, labels in loader:
            outputs = model(features.to(device))
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()

    accuracy = correct / total
    return accuracy, correct, total

In [6]:
def train(model, train_loader, test_loader, criterion, optimizer, num_epochs=50, debug=False):
    best_accuracy, best_epoch = 0, 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * features.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy, correct, total = get_metrics(model, test_loader)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_epoch = epoch
            # torch.save(model.state_dict(), f'adhd_model_{epoch}.pth')
        if debug: print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, Total Test Samples: {total}, Correct Predictions: {correct}')

    # Save the model
    # torch.save(model.state_dict(), 'adhd_model.pth')
    if debug: print(f'Best Accuracy: {best_accuracy:.4f} at Epoch [{best_epoch+1}]')

    return best_accuracy, best_epoch+1

In [7]:
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

fig = go.Figure()
model_results = {}

# Define hyperparameters
input_size = len(feature_columns)  # Number of features

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

for num_layers in [2, 3, 4, 5]:
    for hidden_size in [16, 32, 64, 128]:
        # hidden_size = 32
        # num_layers = 3
        results = {}
        for test_size in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:
            for random_state in tqdm(range(10), desc=f'Num Layers: {num_layers}, Hidden Size: {hidden_size}, Test Size: {test_size}'.ljust(55), ncols=100):
                
                model = ADHDModel(input_size, hidden_size, num_layers)

                # Define loss and optimizer
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=0.001)
                
                train_df, test_df = get_split_df(df, test_size=test_size, random_state=random_state)
                train_loader, test_loader = get_dataloaders(train_df, test_df, feature_columns)
                best_accuracy, best_epoch = train(model, train_loader, test_loader, criterion, optimizer)
                results[test_size] = results.get(test_size, []) + [(best_accuracy, best_epoch)]
                # print(f'Test Size: {test_size}, Random State: {random_state}, Best Accuracy: {best_accuracy:.4f}, Best Epoch: {best_epoch}')

        model_results[(num_layers, hidden_size)] = results

        # Plot a line graph of test accuracy vs. test size
        test_sizes = list(results.keys())
        accuracies = [[trial[0] for trial in trials] for trials in results.values()]

        mean_accuracies = [np.mean(acc) for acc in accuracies]
        std_errors = [np.std(acc) for acc in accuracies]

        # Plot using plotly
        fig.add_trace(go.Scatter(
            x=test_sizes,
            y=mean_accuracies,
            mode='lines+markers',
            name=f'{num_layers} Layers, {hidden_size} Hidden Size',
            error_y=dict(
                type='data',
                array=std_errors,
                visible=True
            )
        ))

# Update layout for better visualization (optional)
fig.update_layout(
    title='Model Accuracy vs Test Sizes',
    xaxis_title='Test Sizes',
    yaxis_title='Accuracy',
    legend_title='Legend'
)

fig.show()


Using device: cpu


Num Layers: 2, Hidden Size: 16, Test Size: 0.1         : 100%|██████| 10/10 [00:36<00:00,  3.60s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.15        : 100%|██████| 10/10 [00:32<00:00,  3.20s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.2         : 100%|██████| 10/10 [00:32<00:00,  3.27s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.25        : 100%|██████| 10/10 [00:31<00:00,  3.17s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.3         : 100%|██████| 10/10 [00:30<00:00,  3.06s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.35        : 100%|██████| 10/10 [00:29<00:00,  2.92s/it]
Num Layers: 2, Hidden Size: 16, Test Size: 0.4         : 100%|██████| 10/10 [00:21<00:00,  2.13s/it]
Num Layers: 2, Hidden Size: 32, Test Size: 0.1         : 100%|██████| 10/10 [00:34<00:00,  3.43s/it]
Num Layers: 2, Hidden Size: 32, Test Size: 0.15        : 100%|██████| 10/10 [00:34<00:00,  3.45s/it]
Num Layers: 2, Hidden Size: 32, Test Size: 0.2         : 100%|██████| 10/10 [00:33<00:00,  

In [42]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import numpy as np
import json

fig = go.Figure()

with open('lstm_model_results.json', 'r') as f:
    model_results = json.load(f)

for classifier, results in model_results.items():
    if classifier not in ['(2, 128)', '(3, 128)', '(4, 128)', '(5, 64)']: continue
    l, h = eval(classifier)
    label = f'{l}L, {h}H'
    # Plot a line graph of test accuracy vs. test size
    test_sizes = list(results.keys())
    accuracies = [[trial[0] for trial in trials] for trials in results.values()]

    mean_accuracies = [np.mean(acc) for acc in accuracies]
    std_errors = [np.std(acc) for acc in accuracies]

    # Plot using plotly
    fig.add_trace(go.Scatter(
        x=test_sizes,
        y=mean_accuracies,
        mode='lines+markers',
        # ['Random Forest', 'XGBoost', 'Logistic Regression']
        name=label,
        error_y=dict(
            type='data',
            array=std_errors,
            visible=True,
            thickness=1,
            width=20,
        ),
        line=dict(
            width=3,  # increase line width
        ),
        marker=dict(
            size=10,  # increase marker size
        )
    ))

# Update layout for better visualization (optional)
fig.update_layout(
    title=dict(text='LSTM Models Accuracy vs Split Size', font=dict(size=30, color='black'),
        x=0.5,  # Center the title
        xanchor='center'),
    xaxis_title=dict(text='Validation Set Split Size', font=dict(size=25, color='black')),
    yaxis_title=dict(text='Classification Accuracy', font=dict(size=25, color='black')),
    # legend_title='Legend',
    legend=dict(
        orientation='h',  # horizontal legend
        yanchor='bottom',  # anchor legend to the bottom
        y=1.02,  # position the legend slightly below the plot
        xanchor='right',
        x=1,
        font=dict(size=20, color='black')
    )
)

fig.update_xaxes(tickfont=dict(size=20, color='black'))
fig.update_yaxes(tickfont=dict(size=20, color='black'), dtick=0.1)

# Save the figure with high DPI and tight bounding box
filename = 'result_images/lstm_model_accuracy_vs_test_sizes_ml.png'
pio.write_image(fig, filename, format='png', scale=3)


fig.show()


In [41]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import numpy as np
import json

fig = go.Figure()

with open('ml_model_results.json', 'r') as f:
    model_results = json.load(f)

model_results = {'LR': v for k, v in model_results.items() if 'logistic' in k.lower()}

with open('lstm_model_results.json', 'r') as f:
    lstm_model_results = json.load(f)

model_results['LSTM 5L, 64H'] = lstm_model_results['(5, 64)']

for classifier, results in model_results.items():
    # Plot a line graph of test accuracy vs. test size
    test_sizes = list(results.keys())
    accuracies = [[trial[0] for trial in trials] for trials in results.values()]

    mean_accuracies = [np.mean(acc) for acc in accuracies]
    std_errors = [np.std(acc) for acc in accuracies]

    # Plot using plotly
    fig.add_trace(go.Scatter(
        x=test_sizes,
        y=mean_accuracies,
        mode='lines+markers',
        # ['Random Forest', 'XGBoost', 'Logistic Regression']
        name=classifier,
        error_y=dict(
            type='data',
            array=std_errors,
            visible=True,
            thickness=1,
            width=20,
        ),
        line=dict(
            width=3,  # increase line width
        ),
        marker=dict(
            size=10,  # increase marker size
        )
    ))

# Update layout for better visualization (optional)
fig.update_layout(
    title=dict(text='Best Models Accuracy vs Split Size', font=dict(size=30, color='black'),
        x=0.5,  # Center the title
        xanchor='center'),
    xaxis_title=dict(text='Validation Set Split Size', font=dict(size=25, color='black')),
    yaxis_title=dict(text='Classification Accuracy', font=dict(size=25, color='black')),
    # legend_title='Legend',
    legend=dict(
        orientation='h',  # horizontal legend
        yanchor='bottom',  # anchor legend to the bottom
        y=1.02,  # position the legend slightly below the plot
        xanchor='right',
        x=1,
        font=dict(size=20, color='black')
    )
)

fig.update_xaxes(tickfont=dict(size=20, color='black'))
fig.update_yaxes(tickfont=dict(size=20, color='black'), dtick=0.1)

# Save the figure with high DPI and tight bounding box
filename = 'result_images/best_model_accuracy_vs_test_sizes_ml.png'
pio.write_image(fig, filename, format='png', scale=3)


fig.show()


In [8]:
import json

# Convert tuple keys to strings
model_results_str_keys = {str(k): v for k, v in model_results.items()}

# Save the modified dictionary to a JSON file
with open('lstm_model_results.json', 'w') as f:
    json.dump(model_results_str_keys, f)

### LR, RF and XGB

In [34]:
grouped_df = df.groupby('patient_id').agg({
    'Age': 'first',
    'Handedness': 'first',
    'Sex': 'first',
    'TragetSlide.RT': ['mean', 'std', 'min', 'max', 'median'],
    'TargetSlide.ACC': 'mean',
    'GROUP': 'first'
}).reset_index()

grouped_df.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in grouped_df.columns.values]
grouped_df['Age_first'] = (grouped_df['Age_first'] - grouped_df['Age_first'].mean()) / grouped_df['Age_first'].std()

grouped_df.head()

,patient_id_,Age_first,Handedness_first,Sex_first,TragetSlide.RT_mean,TragetSlide.RT_std,TragetSlide.RT_min,TragetSlide.RT_max,TragetSlide.RT_median,TargetSlide.ACC_mean,GROUP_first
0,102700,0.280760,0,0,-0.559167,0.334253,-1.023885,0.943756,-0.617452,0.9000,0
1,112601,-0.066848,0,1,0.257704,0.727930,-0.987895,2.440953,0.164052,0.8250,1
2,113010,1.323583,1,0,-0.196269,0.419713,-0.761670,1.582327,-0.272716,0.9625,0
3,122000,-2.152493,0,0,-0.085721,0.612392,-0.888665,2.869751,-0.246495,0.9750,1
4,122401,-0.762063,0,1,-0.440630,0.369062,-0.965273,0.797738,-0.525420,0.9875,0


In [35]:
# Define features and labels, then split the data, then apply random forest classifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

feature_columns = ['Age_first', 'Handedness_first', 'Sex_first', 'TragetSlide.RT_mean', 'TragetSlide.RT_std', 'TragetSlide.RT_min', 'TragetSlide.RT_max', 'TragetSlide.RT_median', 'TargetSlide.ACC_mean']
label_columns = ['GROUP_first']

features = grouped_df[feature_columns]
labels = grouped_df[label_columns]



In [43]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

fig = go.Figure()
model_results = {}

param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200]
}

param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': [None, 'balanced']
}

param_grid_xg = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
}

for classifier, param_grid, name in zip([RandomForestClassifier, XGBClassifier, LogisticRegression], 
                                        [param_grid_rf, param_grid_xg, param_grid_lr], 
                                        ['Random Forest', 'XGBoost', 'Logistic Regression']):
    results = {}
    for test_size in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:
        for random_state in tqdm(range(10), desc=f'Classifier: {name}, Test Size: {test_size}'.ljust(50), ncols=100):
            train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=test_size, random_state=random_state)
            
            model = classifier(random_state=0)

            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
            grid_search.fit(train_features, train_labels.values.ravel())

            # Calculate accuracy on test set of the best model
            best_model = grid_search.best_estimator_
            test_predictions = best_model.predict(test_features)
            test_accuracy = accuracy_score(test_labels, test_predictions)

            results[test_size] = results.get(test_size, []) + [(test_accuracy, -1)]

    model_results[str(classifier)] = results

    # Plot a line graph of test accuracy vs. test size
    test_sizes = list(results.keys())
    accuracies = [[trial[0] for trial in trials] for trials in results.values()]

    mean_accuracies = [np.mean(acc) for acc in accuracies]
    std_errors = [np.std(acc) for acc in accuracies]

    # Plot using plotly
    fig.add_trace(go.Scatter(
        x=test_sizes,
        y=mean_accuracies,
        mode='lines+markers',
        name=name,
        error_y=dict(
            type='data',
            array=std_errors,
            visible=True
        )
    ))

# Update layout for better visualization (optional)
fig.update_layout(
    title='Model Accuracy vs Test Sizes',
    xaxis_title='Test Sizes',
    yaxis_title='Accuracy',
    legend_title='Legend'
)

fig.show()


Classifier: Random Forest, Test Size: 0.1         : 100%|███████████| 10/10 [00:11<00:00,  1.12s/it]
Classifier: Random Forest, Test Size: 0.15        : 100%|███████████| 10/10 [00:11<00:00,  1.10s/it]
Classifier: Random Forest, Test Size: 0.2         : 100%|███████████| 10/10 [00:10<00:00,  1.03s/it]
Classifier: Random Forest, Test Size: 0.25        : 100%|███████████| 10/10 [00:10<00:00,  1.03s/it]
Classifier: Random Forest, Test Size: 0.3         : 100%|███████████| 10/10 [00:10<00:00,  1.03s/it]
Classifier: Random Forest, Test Size: 0.35        : 100%|███████████| 10/10 [00:10<00:00,  1.06s/it]
Classifier: Random Forest, Test Size: 0.4         : 100%|███████████| 10/10 [00:10<00:00,  1.03s/it]
Classifier: XGBoost, Test Size: 0.1               : 100%|███████████| 10/10 [00:01<00:00,  7.95it/s]
Classifier: XGBoost, Test Size: 0.15              : 100%|███████████| 10/10 [00:01<00:00,  7.00it/s]
Classifier: XGBoost, Test Size: 0.2               : 100%|███████████| 10/10 [00:01<00:00,  

In [44]:
import json

# Convert tuple keys to strings
model_results_str_keys = {str(k): v for k, v in model_results.items()}

# Save the modified dictionary to a JSON file
with open('ml_model_results.json', 'w') as f:
    json.dump(model_results_str_keys, f)

In [30]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import numpy as np

fig = go.Figure()
with open('ml_model_results.json', 'r') as f:
    model_results = json.load(f)

with open('lstm_model_results.json', 'r') as f:
    lstm_model_results = json.load(f)

for classifier, results in model_results.items():
    # Plot a line graph of test accuracy vs. test size
    test_sizes = list(results.keys())
    accuracies = [[trial[0] for trial in trials] for trials in results.values()]

    mean_accuracies = [np.mean(acc) for acc in accuracies]
    std_errors = [np.std(acc) for acc in accuracies]

    # Plot using plotly
    fig.add_trace(go.Scatter(
        x=test_sizes,
        y=mean_accuracies,
        mode='lines+markers',
        # ['Random Forest', 'XGBoost', 'Logistic Regression']
        name='RF' if 'Random' in classifier else 'XGB' if 'XGB' in classifier else 'LR',
        error_y=dict(
            type='data',
            array=std_errors,
            visible=True,
            thickness=1,
            width=20,
        ),
        line=dict(
            width=3,  # increase line width
        ),
        marker=dict(
            size=10,  # increase marker size
        )
    ))

# Update layout for better visualization (optional)
fig.update_layout(
    title=dict(text='ML Models Accuracy vs Split Size', font=dict(size=30, color='black'),
        x=0.5,  # Center the title
        xanchor='center'),
    xaxis_title=dict(text='Validation Set Split Size', font=dict(size=25, color='black')),
    yaxis_title=dict(text='Classification Accuracy', font=dict(size=25, color='black')),
    # legend_title='Legend',
    legend=dict(
        orientation='h',  # horizontal legend
        yanchor='bottom',  # anchor legend to the bottom
        y=1.02,  # position the legend slightly below the plot
        xanchor='right',
        x=1,
        font=dict(size=20, color='black')
    )
)

fig.update_xaxes(tickfont=dict(size=20, color='black'))
fig.update_yaxes(tickfont=dict(size=20, color='black'))

# Save the figure with high DPI and tight bounding box
filename = 'result_images/model_accuracy_vs_test_sizes_ml.png'
pio.write_image(fig, filename, format='png', scale=3)


fig.show()
